In [1]:
#All the import
import gradio as gr
import datetime
import uuid
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.document_loaders import NotionDirectoryLoader, NotionDBLoader
from pydantic import BaseModel
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langgraph.graph import MessagesState
from langchain_core.messages import SystemMessage, HumanMessage, RemoveMessage, AIMessage
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
from langgraph.graph import MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import tools_condition, ToolNode

d:\Tensorflow\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
embedder = NVIDIAEmbeddings(
  model="nvidia/nv-embedqa-mistral-7b-v2", 
  api_key="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", 
  truncate="NONE", 
  )

# intruct_model = ChatNVIDIA(
#   model="mistralai/mistral-small-24b-instruct",
#   api_key="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", 
#   temperature=0.2,
#   top_p=0.7,
#   max_tokens=1024,
# )


client = ChatNVIDIA(
  model="meta/llama-3.3-70b-instruct",
  api_key="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", 
  temperature=0.2,
  top_p=0.7,
  max_tokens=1024,
)

# client = ChatNVIDIA(
#   model="meta/llama-3.1-8b-instruct",
#   api_key="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", 
#   temperature=0.2,
#   top_p=0.7,
#   max_tokens=1024,
# )

In [3]:
loader = NotionDBLoader(
    integration_token="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",
    database_id="192dbce6201c80e490b9df38dd0af15c",
    request_timeout_sec=30,  # optional, defaults to 10
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", ";", ",", " ", ""],
)

def include_doc(doc):
     ## Some chunks will be overburdened with useless numerical data, so we'll filter it out
     string = doc.page_content
     if len([l for l in string if l.isalpha()]) < (len(string)//2):
         return False
     return True
    
## Some nice custom preprocessing
docs_split = text_splitter.split_documents(docs)
docs_split = [doc for doc in docs_split if include_doc(doc)]
convstore = FAISS.from_documents(docs_split, embedding=embedder)
retriever = convstore.as_retriever()

In [ ]:
# Function from this block is created by Gemini, doesn't work, prob cuz we only using database and not a page type so dont hav real Page ID to modify database content.
from notion_client import Client
import os

# --- Initialization ---
# It's better practice to load secrets from environment variables
# For example: notion_token = os.getenv("NOTION_TOKEN")
# Using the provided token directly for this example:
notion_token = "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"
if not notion_token:
    raise ValueError("Notion API token not found. Set NOTION_TOKEN environment variable or add it directly.")

notion = Client(auth=notion_token)
# Your specific Notion Database ID
database_id = "192dbce6201c80e490b9df38dd0af15c"

def update_task_status(task_name: str, new_status: str) -> str:
    """
    Updates the status of a task in the specified Notion database by its exact name.

    Args:
        task_name: The exact name of the task (title property) to update.
        new_status: The new status value (must be a valid option in Notion).

    Returns:
        A string indicating success or failure, including potential error details.
    """
    # --- Configuration ---
    # !! IMPORTANT !!: Verify these property names match your Notion database exactly.
    title_property_name = "Task Names" # The name of your "Title" property column
    status_property_name = "Status Update" # The name of your "Status" property column

    try:
        # 1. Query for the page using the exact task name in the title property
        query_filter = {
            "property": title_property_name,
            "title": {"equals": task_name}
        }
        results = notion.databases.query(
            database_id=database_id,
            filter=query_filter,
        )

        # Check if any results were found
        if not results or not results.get("results"):
            # You could add a fallback search using "contains" here if desired
            return f"Error: No task found with the exact name: '{task_name}' in property '{title_property_name}'."

        # Handle cases where the exact name matches multiple pages (should be rare for titles)
        if len(results["results"]) > 1:
             return f"Error: Multiple tasks found with the exact name '{task_name}'. Please ensure task names are unique."

        # 2. Get the Page ID from the (first and only) query result
        page_id_to_update = results["results"][0]["id"]

        # 3. Prepare the properties payload for the update
        properties_to_update = {
            status_property_name: {
                "status": {
                    "name": new_status # This name MUST match a valid status option in Notion
                }
            }
        }

        # 4. Execute the page update
        notion.pages.update(
            page_id=page_id_to_update, # Use the dynamically found page_id
            properties=properties_to_update
        )

        return f"Success: Task '{task_name}' (Page ID: {page_id_to_update}) status updated to '{new_status}' in property '{status_property_name}'."

    except Exception as e:
        # Catch potential API errors and provide informative feedback
        error_message = str(e)
        print(f"Notion API Error attempting to update task '{task_name}' to status '{new_status}': {error_message}")

        # Try to give more specific feedback based on common Notion API errors
        if "validation failed" in error_message.lower() or "is not a valid option" in error_message.lower():
             return f"Failed to update task '{task_name}'. Error: '{new_status}' is likely an invalid status option for the '{status_property_name}' property in Notion. Please use a valid status name."
        elif "could not find page" in error_message.lower():
             # This might happen if the page was deleted between query and update
             return f"Failed to update task '{task_name}'. Error: Could not find the page (ID: {page_id_to_update}) just before updating. It might have been deleted."
        elif "permission" in error_message.lower():
             return f"Failed to update task '{task_name}'. Error: The Notion integration token lacks permission to edit pages in database ID '{database_id}'."
        else:
             # General error catch-all
             return f"Failed to update task '{task_name}'. An unexpected Notion API error occurred: {error_message}"

# --- Tool Binding Example (ensure 'client' is your LLM instance) ---
# Assuming 'client' is your ChatNVIDIA or similar LangChain LLM client instance
tools = [update_task_status]
client_tools = client.bind_tools(tools)

# --- Direct Call Example (for testing) ---
# if __name__ == "__main__":
#     # Replace with a real task name from your DB and a VALID status option
#     test_task = "Information on Machine Learning"
#     valid_status = "In Progress"
#     invalid_status = "Totally Finished" # Assume this is NOT a valid option

In [5]:
class State(MessagesState):
    summary: str

def Normal(state: State):
    
    # Get summary if it exists
    summary = state.get("summary", "")

    # If there is summary, then we add it
    if summary:
        
        # Add summary to system message
        system_message = f"Summary of conversation earlier. While u cannot confirm any of the AI responses, assume that they are all correct {summary}"

        # Append summary to any newer messages
        messages = [SystemMessage(content=system_message)] + state["messages"]
    
    else:
        messages = state["messages"]
        
    response = client.invoke(state["messages"])
    return {
        "messages": response
    }

def Notiondb(state: State):
    
    # Get summary if it exists
    summary = state.get("summary", "")

    # If there is summary, then we add it
    if summary:
        
        # Add summary to system message
        system_message = f"Summary of conversation earlier: {summary}"

        # Append summary to any newer messages
        messages = [SystemMessage(content=system_message)] + state["messages"]
    
    else:
        messages = state["messages"]
        
    formating = """
    You be given a information on a few documents relating to the user query.
    If user is asking information about one specific document then only give them information on that one document only. Do
    not use other documents information
    If the user is asking information about a collection of doucments then you can decide which documents user is asking for and
    use multiple documents to answer the query
    Be concise and get to the point with your answer. Then ask user if there's anything else they need
    You can also update the Notion Database
    """

    current_time = datetime.datetime.now().isoformat()
    relevant_doc = retriever.invoke(state["messages"][-1].content)
    data = [info.metadata for info in relevant_doc]
    all_metadata = [doc.metadata for doc in docs]
    context = "\n\n".join(
    f"This part belongs to the document {doc.metadata['task names']}\n\n{doc.page_content}"
    for doc in relevant_doc
    )
    query = str(current_time) + str(all_metadata) + context + "Answer the questions using information provided" + formating
    messages = state["messages"] + [HumanMessage(content=query)]
    answer = client_tools.invoke(messages)
    return  {
        "messages": answer
    }

def DecideFunction(state: State):
    instruction ="""
    Determine if user is asking/talking about general stuff or information relating to the task/documents in Notion Database. 
    You have been given a list of all the document's name from the database.
    Answer in 0 if the document or task the user is asking about is in the documents list or they mention the notion database specifically
    Only answer in 1 if the user is just making conversation.
    Make sure u only return the number either 0 or 1.
    """

    last_message = state["messages"][-1].content
    all_documents = [doc.metadata["task names"] for doc in docs]
    query = last_message + "Document names:" + str(all_documents) + instruction
    decision = client.invoke(query)
    
    if decision.content == "0":
        return "Notiondb"
    return "Normal"

#Function copied from LangGraph course
def summarize_conversation(state: State):
    
    # First, we get any existing summary
    summary = state.get("summary", "")

    # Create our summarization prompt 
    if summary:
        
        # A summary already exists
        summary_message = (
            f"This is summary of the conversation to date: {summary}\n\n"
            """
            You are a chatbot and have been conversing with the user.
            Now extend the summary by taking into account the new messages above
            Do not worry about the information accuracy or anything.
            While u cannot confirm any of the AI responses, assume that they are all correct and include them in the summerisation
            Your job is to literally summerise the conversation and nothing else
            """
        )
        
    else:
        summary_message = """
        You are a chatbot and have been conversing with the user.
        Now create a summary of the conversation, do not worry about the information accuracy or anything.
        While u cannot confirm any of the AI responses, assume that they are all correct and include them in the summerisation
        Your job is to literally summerise the conversation and nothing else."""

    # Add prompt to our history
    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = client.invoke(messages)
    print(response)
    
    # Delete all but the 2 most recent messages
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    return {"summary": response.content, "messages": delete_messages}

# Function copied from LangGraph course
def should_continue(state: State):
    
    """Return the next node to execute."""
    
    messages = state["messages"]
    
    # If there are more than six messages, then we summarize the conversation
    if len(messages) > 6:
        return "summarize_conversation"
    
    # Otherwise we can just end
    return END

builder = StateGraph(State)
builder.add_node("Normal", Normal)
builder.add_node("Notiondb", Notiondb)
builder.add_node(summarize_conversation)
builder.add_node("tools", ToolNode(tools))

builder.add_conditional_edges(START, DecideFunction, ["Normal", "Notiondb"])
builder.add_conditional_edges("Normal", should_continue)
builder.add_conditional_edges("Notiondb", should_continue)
builder.add_conditional_edges("Notiondb", tools_condition)
builder.add_edge("tools", "Notiondb")
builder.add_edge("summarize_conversation", END)

# Add
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

# View
graph.get_graph().print_ascii()
#display(Image(graph.get_graph().draw_mermaid_png()))

         +-----------+           
         | __start__ |           
         +-----------+           
          ..        ..           
        ..            ..         
       .                .        
+--------+          +----------+ 
| Normal |          | Notiondb | 
+--------+        ..+----------+ 
      *       ....        *      
      *    ...            *      
      *  ..               *      
+---------+          +-------+   
| __end__ |          | tools |   
+---------+          +-------+   


In [6]:
# Create a thread
config = {"configurable": {"thread_id": "1"}}

# Start conversation
input_message = HumanMessage(content="Yo, what's up pookie")
output = graph.invoke({"messages": [input_message]}, config) 
for m in output['messages']:
    m.pretty_print()

================================ Human Message =================================

Yo, what's up pookie
================================== Ai Message ==================================

Not much, just hanging out in the digital realm! It's great to chat with you, though! What's on your mind? Want to talk about something in particular or just shoot the breeze?


In [7]:
input_message = HumanMessage(content="What's the due date of the Information on Machine Learning task?")
output = graph.invoke({"messages": [input_message]}, config) 
for m in output['messages']:
    m.pretty_print()

================================ Human Message =================================

Yo, what's up pookie
================================== Ai Message ==================================

Not much, just hanging out in the digital realm! It's great to chat with you, though! What's on your mind? Want to talk about something in particular or just shoot the breeze?
================================ Human Message =================================

What's the due date of the Information on Machine Learning task?
================================== Ai Message ==================================

The due date of the "Information on Machine Learning" task is 2025-06-18. 

Is there anything else you need?


In [8]:
input_message = HumanMessage(content="Yea can u remind me of the content inside the Business of Music document?")
output = graph.invoke({"messages": [input_message]}, config) 
for m in output['messages']:
    m.pretty_print()

================================ Human Message =================================

Yo, what's up pookie
================================== Ai Message ==================================

Not much, just hanging out in the digital realm! It's great to chat with you, though! What's on your mind? Want to talk about something in particular or just shoot the breeze?
================================ Human Message =================================

What's the due date of the Information on Machine Learning task?
================================== Ai Message ==================================

The due date of the "Information on Machine Learning" task is 2025-06-18. 

Is there anything else you need?
================================ Human Message =================================

Yea can u remind me of the content inside the Business of Music document?
================================== Ai Message ==================================

The content inside the Business of Music document includes an i

In [9]:
input_message = HumanMessage(content="Can u tell me all task assigned to bike in the Notion Database?")
output = graph.invoke({"messages": [input_message]}, config) 
for m in output['messages']:
    m.pretty_print()

content='The conversation started with a casual greeting. The user then asked about the due date of the "Information on Machine Learning" task, which was stated to be 2025-06-18. The user also inquired about the content of the "Business of Music" document, which was said to include topics such as the ethics and economic impact of AI in music production, the history of the recording music industry, and the evolution of techniques and tools used by businesses. Additionally, the user asked about tasks assigned to "bike" in the Notion Database, which included tasks like "Cyber report", "ML Workshop", "Game Design", "NNO CUP RULES", and "Information on Machine Learning" with their respective due dates.' additional_kwargs={} response_metadata={'role': 'assistant', 'content': 'The conversation started with a casual greeting. The user then asked about the due date of the "Information on Machine Learning" task, which was stated to be 2025-06-18. The user also inquired about the content of the "

In [10]:
input_message = HumanMessage(content="Can you give me all status updates of all task in the Notion Database")
output = graph.invoke({"messages": [input_message]}, config) 
for m in output['messages']:
    m.pretty_print()

================================ Human Message =================================

Can u tell me all task assigned to bike in the Notion Database?
================================== Ai Message ==================================

The tasks assigned to "bike" in the Notion Database are:

1. Cyber report (due date: 2025-09-22)
2. ML Workshop (due date: 2025-05-09)
3. Game Design (due date: 2025-05-18)
4. NNO CUP RULES (due date: 2025-08-20)
5. Information on Machine Learning (due date: 2025-06-18)

Is there anything else you need?
================================ Human Message =================================

Can you give me all status updates of all task in the Notion Database
================================== Ai Message ==================================

I don't have have access to a Notion Database, and I didn't receive any information about the tasks or their status updates. My previous response was an error.

If you'd like to share the information about the tasks and their status 

In [11]:
input_message = HumanMessage(content="Can you mark the Business of Music document task as done")
output = graph.invoke({"messages": [input_message]}, config) 
for m in output['messages']:
    m.pretty_print()

content='Here is a summary of the conversation to date:\n\nThe conversation started with a casual greeting. The user then asked about tasks assigned to "bike" in the Notion Database, and was informed that the tasks included "Cyber report", "ML Workshop", "Game Design", "NNO CUP RULES", and "Information on Machine Learning" with their respective due dates. The user then asked for all status updates of all tasks in the Notion Database, but no information was provided. The user then asked to mark the "Business of Music" document task as done, and was informed that the task status was updated to "Done". The user also inquired about the content of the "Business of Music" document, which was said to include topics such as the ethics and economic impact of AI in music production. \n\nNote that the initial summary provided by the user was not entirely accurate, as it mentioned the due date of the "Information on Machine Learning" task, which was not discussed in the conversation. However, the 

In [12]:
graph.get_state(config).values.get("summary","")

'Here is a summary of the conversation to date:\n\nThe conversation started with a casual greeting. The user then asked about tasks assigned to "bike" in the Notion Database, and was informed that the tasks included "Cyber report", "ML Workshop", "Game Design", "NNO CUP RULES", and "Information on Machine Learning" with their respective due dates. The user then asked for all status updates of all tasks in the Notion Database, but no information was provided. The user then asked to mark the "Business of Music" document task as done, and was informed that the task status was updated to "Done". The user also inquired about the content of the "Business of Music" document, which was said to include topics such as the ethics and economic impact of AI in music production. \n\nNote that the initial summary provided by the user was not entirely accurate, as it mentioned the due date of the "Information on Machine Learning" task, which was not discussed in the conversation. However, the rest of 

In [13]:
input_message = HumanMessage(content="What is the status update for the Business of Music task?")
output = graph.invoke({"messages": [input_message]}, config) 
for m in output['messages']:
    m.pretty_print()

================================= Tool Message =================================
Name: update_task_status

Success: Task 'Business of Music' (Page ID: 19cdbce6-201c-80c3-a97c-e518268fc67d) status updated to 'Done' in property 'Status Update'.
================================== Ai Message ==================================

The document "Business of Music" has the following information:

The ethics and economic impact surrounding the Integration of AI in music production and creativity in the recorded music industry. The music industry has consistently needed to adapt to new technologies and implement new ways in which music is produced, consumed and marketed. The newest challenge currently faced by the industry is the creation and usage of AI song writing tools, specifically the copyright laws surrounding it and the economic and ethical implications it brings to music creators.

Is there anything else you need? 

<function@update_task_status>{"task_name": "Business of Music", "new_stat

In [14]:
#Code below was generated entirely by Gemini code for deployment phase
# Assuming 'graph' is your compiled LangGraph with MemorySaver from previous cells
# Assuming 'State' class is defined

# --- Keep your LangGraph setup code above this ---

# Corrected Gradio Interface Function
def chat_interface(message, history, thread_id_state):
    # Ensure a thread_id exists for the session, create if not
    if not thread_id_state:
        thread_id_state = "thread_" + str(uuid.uuid4())
        print(f"New session started with thread_id: {thread_id_state}") # For debugging

    config = {"configurable": {"thread_id": thread_id_state}}

    # Prepare the input for the graph - just the new user message
    input_message = HumanMessage(content=message)
    input_data = {"messages": [input_message]}

    # Invoke the graph - MemorySaver loads history based on thread_id
    # Pass config using the 'config' keyword argument
    output_state = graph.invoke(input_data, config=config)

    # The output_state['messages'] contains the updated full history
    # We need the last message which should be the AI's response
    ai_response_content = ""
    if output_state and "messages" in output_state and output_state["messages"]:
        last_message = output_state["messages"][-1]
        if isinstance(last_message, AIMessage):
            ai_response_content = last_message.content
        else:
            # Handle cases where the last message might not be AI (e.g., error, graph structure)
            print("Warning: Last message was not from AI:", last_message)
            ai_response_content = "Sorry, I encountered an issue processing that."

    # Update Gradio's history (list of lists)
    # The 'history' variable holds the previous turns [[user, ai], [user, ai]]
    updated_gradio_history = history + [[message, ai_response_content]]

    # Return the updated Gradio history and the persistent thread_id
    return updated_gradio_history, updated_gradio_history, thread_id_state

# Gradio blocks app
with gr.Blocks() as demo:
    gr.Markdown("## 🧠 LangGraph Chatbot Interface")

    # Chatbot component to display conversation
    chatbot = gr.Chatbot(label="Your Assistant")

    # Textbox for user input
    msg = gr.Textbox(label="Enter your message")

    # State to store Gradio's view of history (list of lists) for display
    history_state = gr.State([])

    # State to store the LangGraph thread_id across interactions for this session
    thread_id_state = gr.State(None) # Initialize as None

    # Function to clear the textbox after submission
    def clear_textbox():
        return gr.Textbox(value="")

    # Connect the submit action (when user presses Enter or clicks Submit)
    msg.submit(
        chat_interface,
        # Inputs: current message, Gradio history, LangGraph thread_id
        inputs=[msg, history_state, thread_id_state],
        # Outputs: updated chatbot display, updated Gradio history, updated thread_id
        outputs=[chatbot, history_state, thread_id_state]
    ).then(
        clear_textbox, # After chat_interface runs, clear the textbox
        inputs=None,
        outputs=msg
    )


# Launch Gradio App
demo.launch()

d:\Tensorflow\Lib\site-packages\gradio\components\chatbot.py:273: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
